In [1]:
using Serialization, Turing
include("ode_problem.jl");
include("target_probability.jl");
include("bayesian_inference.jl");

[ Info: Precompiling Turing [fce5fe82-541a-59a6-adf8-730c64b5f9a0] 
[ Info: Precompiling TuringOptimExt [cd2420fc-8d31-5c39-8d98-0365cfcf7d6e] (cache misses: wrong dep version loaded (4))
[ Info: Precompiling Catalyst [479239e8-5488-4da2-87a7-35f2df7eef83] (cache misses: wrong dep version loaded (4))
[ Info: Precompiling DiffEqBaseTrackerExt [1ba67e42-7aa7-5162-ac9b-c09642cdebbf] (cache misses: wrong dep version loaded (4))
[ Info: Precompiling LinearSolveKernelAbstractionsExt [385e4588-a1a0-5c1d-98fa-d45bf6f8ecf9] (cache misses: wrong dep version loaded (2))
[ Info: Precompiling SimpleNonlinearSolveTrackerExt [ac004708-0488-5d74-bdd8-3e2743d441a0] (cache misses: wrong dep version loaded (4))
[ Info: Precompiling OptimizationMTKExt [ead85033-3460-5ce4-9d4b-429d76e53be9] (cache misses: wrong dep version loaded (4))
[ Info: Precompiling DifferentialEquations [0c46a032-eb83-5123-abaf-570d42b7fbaa] (cache misses: wrong dep version loaded (4))
[ Info: Precompiling AdvancedHMCOrdinaryDiffEqE

### Define Target Distribution for Sampling

In [2]:
#first input we need are data points and each point's associated standard deviation
experiment_timecourse = deserialize("outputs/000_processed_active_G_timecourse.dict")
experiment_dose_response = deserialize("outputs/000_processed_active_G_dose_response.dict");
#reshape for input to target probability 
data_points = vcat(experiment_timecourse["response"], experiment_dose_response["response"])
std_dev = vcat(fill(experiment_timecourse["standard_error"], length(experiment_timecourse["response"])), 
fill(experiment_dose_response["standard_error"], length(experiment_dose_response["response"])))

#next input we need is an ode problem
odesys, u0, tspan, p = return_ode_problem_default_inputs()
odeprob = DifferentialEquations.ODEProblem(odesys, u0, tspan, p);

#also need regularization hyperparameters
#note - must convert from M to molecules to match units
avogadros_constant = 6.022e23 
regularization_hyperparams = Dict("mean" => log10(1.01e-7*avogadros_constant), "std_dev" => 0.8, "lambda" => 1)
regularization_hyperparams_theoretical = Dict("mean" => log10(5.0e-9*avogadros_constant), "std_dev" => 1, "lambda" => 1)

#and ode solver inputs for both timecourse simulation and dose response simulation
solver_inputs_timecourse = return_ode_problem_solver_default_inputs("timecourse")
solver_inputs_dose_response = return_ode_problem_solver_default_inputs("dose_response")

#finally, ligand concentrations for dose respones simulation and normalization
ligand_doses = experiment_dose_response["ligand_stimulation (molecules)"];
normalization_dose = experiment_dose_response["normalize_to_response_at_dose"];
all_ligand_doses = vcat(ligand_doses,normalization_dose);

In [3]:
#target probability definition
target_distribution_regularized = logprob_regularized(data_points, odeprob, std_dev, regularization_hyperparams, solver_inputs_timecourse, 
solver_inputs_dose_response, ligand_doses, normalization_dose);

target_distribution_regularized_theoretical = logprob_regularized(data_points, odeprob, std_dev, regularization_hyperparams_theoretical, solver_inputs_timecourse, 
solver_inputs_dose_response, ligand_doses, normalization_dose);

target_distribution_unregularized = logprob_unregularized(data_points, odeprob, std_dev, solver_inputs_timecourse, 
solver_inputs_dose_response, all_ligand_doses);

### Run Affine Invariant Sampler

In [4]:
using BenchmarkTools
#affine invariant sampler parameters
n_ensemble = 1
n_walkers = 10
n_iterations = 100

#target probability
target_probability = Dict("unregularized"=>target_distribution_unregularized, 
"regularized"=>target_distribution_regularized, "regularized_theoretical"=>target_distribution_regularized_theoretical)

#initial run for both regularized and unregularized 
@btime for i in ["unregularized"]
    affine_invariant_mcmc_firstrun(n_ensemble, n_walkers, n_iterations, target_probability[i], i)
end

  6.310 s (17480702 allocations: 1.10 GiB)


In [ ]:
n_subchains = 50
for i in 1:n_subchains
    affine_invariant_mcmc(n_ensemble, n_walkers, n_iterations, target_probability["unregularized"], i, "unregularized")
end